In [1]:
import geopandas as gpd
import rasterio as rio
from rasterio.plot import show
import numpy as np
from rasterstats import zonal_stats
import GOSTRocks.rasterMisc as rMisc

In [2]:
pluvial_raw = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\RawData\P_1in100(1).tif')
fluvial_raw = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\RawData\FU_1in100(1).tif')

In [3]:
pop_raw = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\RawData\Population\Population\SSD_population_v1_0_gridded.tif')
pop = pop_raw.read(1)

In [4]:
districts_dir = r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\SS_ADM2.shp"
districts = gpd.read_file(districts_dir)

## Pluvial Reclass

In [5]:
pluvial_rc = pluvial_raw.read(1)

In [6]:
pluvial_rc[np.where(pluvial_rc <= 0.5)] = 0
pluvial_rc[np.where(pluvial_rc > 0.5)] = 1
pluvial_rc[np.where(pluvial_rc >= 999)] = 0

In [ ]:
with rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\PluvialUndefendedFathom100_noWater_50cmPlus.tif', 'w',
             driver= pluvial_raw.driver,
             height = pluvial_raw.height,
             width = pluvial_raw.width,
             count = pluvial_raw.count,
             crs = pluvial_raw.crs,
             transform = pluvial_raw.transform,
             dtype = 'float32'
)as dst:
    dst.write(pluvial_rc,1)

## Fluvial Reclass

In [ ]:
fluvial_rc = fluvial_raw.read(1)

In [ ]:
fluvial_rc[np.where(fluvial_rc <= 0.5)] = 0
fluvial_rc[np.where(fluvial_rc > 0.5)] = 1
fluvial_rc[np.where(fluvial_rc >= 999)] = 0

In [ ]:
with rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\FluvialFathom100_noWater_50cmPlus.tif', 'w',
             driver= fluvial_raw.driver,
             height = fluvial_raw.height,
             width = fluvial_raw.width,
             count = fluvial_raw.count,
             crs = fluvial_raw.crs,
             transform = fluvial_raw.transform,
             dtype = 'float32'
)as dst:
    dst.write(fluvial_rc,1)

## Pluvial Data Zonal Statistics,  50cm 

In [ ]:
pluvialmask = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\PluvialUndefendedFathom100_noWater_50cmPlus.tif')

In [ ]:
outputdir = r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\PluvialUndefendedFathom100_noWater_50cmPlus_STD.tif'

In [ ]:
# Make sure that both rasters have the exact same resolution, crs, and number of pixels
rMisc.standardizeInputRasters(pluvialmask, pop_raw, outputdir, data_type="C")

In [ ]:
pluvialmask_STD = rio.open(outputdir)

In [ ]:
pluvialwPop =pop * pluvialmask_STD.read(1)

In [ ]:
pluvialwPop[pluvialwPop< 0]=0 #get turning negative numbers to 0

In [ ]:
np.unique(pluvialwPop)

In [ ]:
pluvialwPop

In [ ]:
with rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\pop_inPluvial50cmPlus.tif', 'w',
             driver = pop_raw.driver,
             height = pop_raw.height,
             width = pop_raw.width,
             count = pop_raw.count,
             crs = pop_raw.crs,
             transform = pop_raw.transform,
             dtype = 'float32'
)as dst:
    dst.write(pluvialwPop,1)

In [ ]:
# Read in again to have variable pointing to the rasterio object
pluvialwPop = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\pop_inPluvial50cmPlus.tif')

In [ ]:
np.unique(pluvialwPop.read(1))

In [ ]:
np.sum(pluvialwPop.read(1))

In [ ]:
affine = pluvialwPop.transform

In [ ]:
sum_pop_plu = zonal_stats(districts, pluvialwPop.read(1), affine=affine, stats="sum")

In [ ]:
sum_pop_plu

In [ ]:
sum_pop_plu_sumONLY = []
for i in sum_pop_plu:
    sum_pop_plu_sumONLY.append(i['sum'])

In [ ]:
sum_pop_plu_sumONLY

In [ ]:
districts['pop_plu50c'] = sum_pop_plu_sumONLY

In [ ]:
districts

## Fluvial Data Zonal Statistics

In [ ]:
fluvialmask = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\FluvialFathom100_noWater_50cmPlus.tif')

In [ ]:
outputdir_f = r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\FluvialFathom100_noWater_50cmPlus_STD.tif'

In [ ]:
# Make sure that both rasters have the exact same resolution, crs, and number of pixels
rMisc.standardizeInputRasters(fluvialmask, pop_raw, outputdir_f, data_type="C")

In [ ]:
fluvialmask_STD = rio.open(outputdir_f)

In [ ]:
fluvialwPop =pop * fluvialmask_STD.read(1)

In [ ]:
fluvialwPop[fluvialwPop< 0]=0 #get turning negative numbers to 0

In [ ]:
with rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\pop_inFluvial50cmPlus.tif', 'w',
             driver= pop_raw.driver,
             height = pop_raw.height,
             width = pop_raw.width,
             count = pop_raw.count,
             crs = pop_raw.crs,
             transform = pop_raw.transform,
             dtype = 'float32'
)as dst:
    dst.write(fluvialwPop,1)

In [ ]:
# Read in again to have variable pointing to the rasterio object
fluvialwPop = rio.open(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\ProcessedFathom\50cmPlus\pop_inFluvial50cmPlus.tif')

In [ ]:
np.unique(fluvialwPop.read(1))

In [ ]:
np.sum(fluvialwPop.read(1))

In [ ]:
affine = fluvialwPop.transform

In [ ]:
sum_pop_flu = zonal_stats(districts, fluvialwPop.read(1), affine=affine, stats="sum")

In [ ]:
sum_pop_flu

In [ ]:
sum_pop_flu_sumONLY = []
for i in sum_pop_flu:
    sum_pop_flu_sumONLY.append(i['sum'])

In [ ]:
sum_pop_flu_sumONLY

In [ ]:
districts['pop_flu50c'] = sum_pop_flu_sumONLY

In [ ]:
districts

## Calculate total population affected by flooding >50cm

In [ ]:
districts['pop_tot50c'] = districts['pop_plu50c'] + districts['pop_flu50c']

In [ ]:
districts

In [ ]:
districts.to_file(r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\SS_ADM2_FathomPvsF_50cm.shp")

# Total % of flooding in [m] disaggregated by flood type

In [35]:
districts_dir2 = r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\SS_ADM2_FathomPvsF_50cm.shp"
districts2 = gpd.read_file(districts_dir2)

In [6]:
pluvial_rc_m = pluvial_raw.read(1)
fluvial_rc_m = fluvial_raw.read(1)

In [7]:
# Only include ares where flooding is more than 0.5m
pluvial_rc_m[np.where(pluvial_rc_m <= 0.5)] = 0

In [8]:
# Only include ares where flooding is more than 0.5m
fluvial_rc_m[np.where(fluvial_rc_m <= 0.5)] = 0

In [10]:
affine = pluvial_raw.transform

In [17]:
dist_wpluvial_m = zonal_stats(districts2, pluvial_rc_m, affine=affine, stats="sum")

C:\Users\glev2\anaconda3\envs\dreamenv\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [18]:
dist_wfluvial_m = zonal_stats(districts2, fluvial_rc_m, affine=affine, stats="sum")

In [19]:
dist_wpluvial_m_sumonly = []
for i in dist_wpluvial_m:
    dist_wpluvial_m_sumonly.append(i['sum'])

In [20]:
dist_wfluvial_m_sumonly = []
for i in dist_wfluvial_m:
    dist_wfluvial_m_sumonly.append(i['sum'])

In [9]:
totalflooding_m = pluvial_rc_m+fluvial_rc_m

In [11]:
dist_wtotalflooding = zonal_stats(districts2, totalflooding_m, affine=affine, stats="sum")

C:\Users\glev2\anaconda3\envs\dreamenv\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [13]:
dist_wtotalflooding_sumonly = []
for i in dist_wtotalflooding:
    dist_wtotalflooding_sumonly.append(i['sum'])

In [15]:
districts2['tot_flood'] = dist_wtotalflooding_sumonly

In [26]:
districts2['tot_plum'] = dist_wpluvial_m_sumonly

In [27]:
districts2['tot_flum'] = dist_wfluvial_m_sumonly

In [28]:
districts2['p_plu50c'] = (districts2['tot_plum']/districts2['tot_flood'])*100

In [29]:
districts2['p_flu50c'] = (districts2['tot_flum']/districts2['tot_flood'])*100

In [41]:
districts2 = districts2.drop(columns=['tot_plum', 'tot_flum', 'tot_flood'])

In [42]:
districts2

,Shape_Leng,Shape_Area,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM1_EN,ADM1_PCODE,ADM0_EN,...,Plu100sum,Plu100mean,Plu100min,Plu100max,pop_plu50c,pop_flu50c,pop_tot50c,p_plu50c,p_flu50c,geometry
0,6.425236,18611.507286,Juba,SS0101,None,None,None,Central Equatoria,SS01,South Sudan,...,2.365945e+06,1.097509,0.0,999.000000,19197.005770,73069.814708,92266.820477,49.646835,50.353165,"POLYGON ((32.10224 5.36788, 32.09834 5.21346, ..."
1,2.082643,2549.814924,Kajo-keji,SS0102,None,None,None,Central Equatoria,SS01,South Sudan,...,5.024976e+04,0.170308,0.0,999.000000,1178.606410,61.659035,1240.265445,68.111614,31.888386,"POLYGON ((31.61854 4.02379, 31.69394 3.95754, ..."
2,3.402761,3545.715110,Lainya,SS0103,None,None,None,Central Equatoria,SS01,South Sudan,...,4.867531e+04,0.118577,0.0,7.426941,3724.817229,957.558599,4682.375829,72.881758,27.118242,"POLYGON ((31.18674 4.30480, 31.17532 4.25454, ..."
3,2.069064,1326.493498,Morobo,SS0104,None,None,None,Central Equatoria,SS01,South Sudan,...,1.762787e+04,0.114901,0.0,10.000000,2812.441371,533.452950,3345.894320,90.147513,9.852487,"POLYGON ((31.04738 3.81817, 31.09567 3.74048, ..."
4,5.221777,10813.630464,Terekeka,SS0105,None,Terkeka,None,Central Equatoria,SS01,South Sudan,...,1.559094e+06,1.242798,0.0,999.000000,1094.880323,7399.032974,8493.913297,46.193880,53.806120,"POLYGON ((31.02171 5.90953, 31.69732 5.87275, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,3.220148,5786.475341,Mvolo,SS1006,None,None,None,Western Equatoria,SS10,South Sudan,...,7.006990e+04,0.104331,0.0,4.371643,1255.328701,530.913422,1786.242124,36.350604,63.649396,"POLYGON ((30.12183 6.35244, 30.15537 6.23657, ..."
74,3.862937,9318.009503,Nagero,SS1007,None,None,None,Western Equatoria,SS10,South Sudan,...,1.288392e+05,0.119003,0.0,6.169495,513.805478,15.457573,529.263051,58.251906,41.748094,"POLYGON ((28.03236 6.74766, 28.04826 6.73971, ..."
75,5.672132,6529.697021,Nzara,SS1008,None,None,None,Western Equatoria,SS10,South Sudan,...,8.990328e+04,0.118757,0.0,6.285339,1201.768536,454.272056,1656.040591,62.497998,37.502002,"POLYGON ((28.52396 6.25181, 28.57275 6.07495, ..."
76,5.860083,12619.589011,Tambura,SS1009,None,None,None,Western Equatoria,SS10,South Sudan,...,1.835113e+05,0.125260,0.0,7.262573,393.224871,151.797714,545.022585,73.412831,26.587169,"POLYGON ((27.06807 6.65988, 27.07929 6.54093, ..."


In [2]:
districts2.to_file(r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme2\SS_ADM2_FathomPvsF_50cm.shp")

NameError: name 'adm2' is not defined